In [2]:
import pandas as pd
import numpy as np
import openpyxl 
import os
import sys
import glob
import regex as re
import warnings
from tqdm.auto import tqdm

# Funzioni

In [3]:
# 1) FILES NELLA CARTELLA 
def Files_in_dir(path, extension = "xlsx"):
    #set working dir
    os.chdir(path)

    #extension = 'xlsx'

    #crea la lista con i nomi di tutti i file nella cartella 
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

    print("Files in Path: " , all_filenames)
    
    return all_filenames

In [4]:
# 2)  Apertura del workbook Excel
def load_workbook(file):
    
    wb = openpyxl.load_workbook(filename= file , data_only=True)
    print("Loaded File: ", file)
        
    return wb

In [13]:
def template_to_df(wb, template, first_row = True):
    if template in wb.sheetnames:
        
        ws = wb[template]
        
        df = pd.DataFrame(ws.values)
        
        if first_row ==True:
            col = list(df.iloc[0, :])
            df = df.iloc[1:,:]
            df.columns = col
        
    return df

In [14]:
def create_finalDf(listaFiles, template):

    finalDf = pd.DataFrame()

    for file in listaFiles:
        print("Extracting Template for File ", file)
        
        try:
            wb = load_workbook(file)

            if template in wb.sheetnames:

                df = template_to_df(wb, template)

                finalDf = pd.concat([finalDf, df], axis=0, ignore_index=True )
        except:
            warnings.warn('Problema con almeno un file')
            print("problem with file ", file)

    return finalDf

In [15]:
def create_sumF100Df(finalDf):
    
    varsToSum = ["Target F100" , "F100 achieved" , "Delta F100"]
    
    for var in varsToSum:
        for riga in range(len(finalDf)):
            value = finalDf.loc[riga, var]
            if type(value) != float and type(value) != int:

                finalDf.loc[riga, var] = np.nan
    
    return finalDf.groupby("File")[varsToSum].sum()
    


In [16]:
#Copia dei parametri in un nuovo File
def copy_to_template(inputDf, targetWb, targetTemplate):
    
    # copia delle colonne nella prima riga del nuovo template
    letter = 1
    for col in inputDf.columns:
        targetWb[targetTemplate].cell(1,letter).value = col
        letter += 1

    # Copia dei valori del dataset nel template
    for riga in range(len(inputDf)):
        for colonna in range(len(inputDf.columns)):
            targetWb[targetTemplate].cell(riga + 2, colonna + 1).value =  inputDf.iloc[riga,colonna]
            
    return targetWb

In [17]:
def apply_styles(wb, template):
    from openpyxl.styles import Font
    from openpyxl.styles import PatternFill
    
    #Bold della prima riga
    BoldFont = Font( bold=True)
    for col in range(wb[template].max_column):
        wb[template].cell(1, col + 1).font = BoldFont
        
    #evidenziare i missing
    '''
    yellow = "00FFFF00"
    MissingFont = Font(color=yellow)
    
    MissingList = ["MISSING", "#DIV/0!",  "#RIF!" ]
    
    for riga in range(wb[template].max_row):
        for colonna in range(wb[template].max_column):
            valore = wb[template].cell(riga+2, colonna +1 ).value
            if valore in MissingList:
                 wb[template].cell(riga+2, colonna +1).fill = PatternFill(start_color=yellow, end_color=yellow, fill_type = "solid")
    '''             
    return wb

# Process for Entire Folder

In [18]:
def get_F100_folder(path, outPath, outFile, template_to_extract = "Parameters Template" ):
             
    #lista files in directory
    filenames = Files_in_dir(path)
        
    # df con tutti i parametri di tutti i fogli
    finalDf = create_finalDf(filenames, template_to_extract)
    
    # df con le somme degli F100
    sumF100Df = create_sumF100Df(finalDf).reset_index()
    
    # inizializzare il nuovo workbook con i parametri
    newWb = openpyxl.Workbook()
    newWb.create_sheet("F100 Sum for File")
    newWb.create_sheet("All Parameters")
    if "Sheet" in newWb.sheetnames:
        del newWb["Sheet"]
        
    # copia dei df nei template nel nuovo wb
    copy_to_template(sumF100Df, newWb, "F100 Sum for File")
    copy_to_template(finalDf, newWb, "All Parameters")
    
    apply_styles(newWb, "F100 Sum for File")
    apply_styles(newWb, "All Parameters")
    
    # salvataggio e close
    newWb.save(outPath + outFile)
    newWb.close()
    

# Applicazione 

In [19]:
path=r"C:\Users\f08132d\OneDrive - CNH Industrial\Desktop\PROGRAMMI SAS\11 TEMPLATE PYTHON\Gruppi FBD - Copia\\"
outPath = r"C:\Users\f08132d\OneDrive - CNH Industrial\Desktop\PROGRAMMI SAS\11 TEMPLATE PYTHON\\"
outFile = r"PV Summary.xlsx"


#run
get_F100_folder(path, outPath, outFile)

Files in Path:  ['CHASSIS my24 .xlsx', 'Copia di ELECTRONICS.xlsx', 'Copia di POWERTRAIN my24.xlsx', 'DRIVELINE my24.xlsx', 'DRIVELINE.xlsx', 'EXTERNAL TRIM.xlsx', 'INTERNAL TRIM  .xlsx', 'IVP Pedal Accelerator.xlsx', 'Lighting from Chiusa.xlsx', 'PNEUMATICS.xlsx', 'POWERTRAIN my24 version1.xlsx', 'POWERTRAIN my24.xlsx', 'reliability_IPU_Nikola from Marseglia.xlsx', 'ZF_reverse_camera_success_run.XLSX']
Extracting Template for File  CHASSIS my24 .xlsx
Loaded File:  CHASSIS my24 .xlsx
Extracting Template for File  Copia di ELECTRONICS.xlsx
Loaded File:  Copia di ELECTRONICS.xlsx
Extracting Template for File  Copia di POWERTRAIN my24.xlsx
problem with file  Copia di POWERTRAIN my24.xlsx
Extracting Template for File  DRIVELINE my24.xlsx


C:\Users\f08132d\AppData\Local\Temp\ipykernel_15508\1437089581.py:17: UserWarning: Problema con almeno un file
  warnings.warn('Problema con almeno un file')


Loaded File:  DRIVELINE my24.xlsx
Extracting Template for File  DRIVELINE.xlsx
Loaded File:  DRIVELINE.xlsx
Extracting Template for File  EXTERNAL TRIM.xlsx
Loaded File:  EXTERNAL TRIM.xlsx
Extracting Template for File  INTERNAL TRIM  .xlsx
Loaded File:  INTERNAL TRIM  .xlsx
Extracting Template for File  IVP Pedal Accelerator.xlsx
Loaded File:  IVP Pedal Accelerator.xlsx
Extracting Template for File  Lighting from Chiusa.xlsx
Loaded File:  Lighting from Chiusa.xlsx
Extracting Template for File  PNEUMATICS.xlsx
Loaded File:  PNEUMATICS.xlsx
Extracting Template for File  POWERTRAIN my24 version1.xlsx
Loaded File:  POWERTRAIN my24 version1.xlsx
Extracting Template for File  POWERTRAIN my24.xlsx
Loaded File:  POWERTRAIN my24.xlsx
Extracting Template for File  reliability_IPU_Nikola from Marseglia.xlsx
Loaded File:  reliability_IPU_Nikola from Marseglia.xlsx
Extracting Template for File  ZF_reverse_camera_success_run.XLSX
Loaded File:  ZF_reverse_camera_success_run.XLSX
